In [1]:
## RAY TO PLANE INTERSECTION BROKEN DOWN
import numpy as np

def ray_to_plane_intersection(u, R, T, l):
    """
    Calculates the intersection of a ray with a plane in a structured light camera system.

    :param u: Image point coordinates as a numpy array [u1, u2, 1].
    :param R: Rotation matrix of the camera.
    :param T: Translation vector of the camera.
    :param l: Vector defining the line on the image plane.

    :return: Intersection point in world coordinates.
    """
    # Converting to homogeneous coordinates
    u = np.append(u, [1])

    # Calculate q (center of projection in world coordinates)
    q = -np.dot(R.T, T)

    # Calculate v (direction vector in world coordinates)
    v = np.dot(R.T, u)

    # Calculate n (normal vector of the plane in world coordinates)
    n = np.dot(R.T, l)

    # Solve for lambda in the plane equation: n^t * (q + lambda * v - q) = 0
    # Simplified to: lambda = -n^t * q / n^t * v
    lambda_val = -np.dot(n.T, q) / np.dot(n.T, v)

    # Calculate the intersection point
    intersection_point = q + lambda_val * v

    return intersection_point

# Example usage
u = np.array([100, 200])  # Example image point coordinates
R = np.eye(3)            # Example rotation matrix (identity matrix)
T = np.array([5, 0, 0]) # Example translation vector
l = np.array([0, 1, -5]) # Example line vector (horizontal line)

# Calculate the intersection point
intersection_point = ray_to_plane_intersection(u, R, T, l)
print(intersection_point)



[-5. -0. -0.]


In [2]:
import cv2 as cv
import numpy as np
import math

def generate_gray_code_patterns(width, height):
    """
    :param width: Projector Width
    :param Height: Projector Height
    """
    # Calculate number of bits required to represent the width
    num_bits = math.ceil(math.log2(width))
    # Calculate the offset to center the pattern
    offset = (2 ** num_bits - width) // 2

    # Initialize pattern storage
    pattern = np.zeros((height, width, num_bits), dtype=np.uint8)
    directory = "C:\\Users\\nludw\\Documents\\Capstone\\Binary Coding\\GrayCodedPictures"  # Change to Pi directory
    # Generate binary and Gray code numbers
    binary_numbers = np.array([list(format(i, '0' + str(num_bits) + 'b')) for i in range(2 ** num_bits)], dtype=np.uint8)
    gray_codes = np.bitwise_xor(binary_numbers[:, :-1], binary_numbers[:, 1:]) #XOR bitwise for gray coding
    gray_codes = np.c_[binary_numbers[:, 0], gray_codes]  # Add the first bit back
    
    # Fill in the pattern
    for i in range(num_bits):
        pattern[:, :, i] = np.tile(gray_codes[(np.arange(width) + offset), i].reshape(1, -1), (height, 1))
        filename = "gray_pattern{}.png".format(i)
        cv.imwrite(directory + "\\" + filename, 255*pattern[:,:,i]) #Need to multiply by 255 for openCV to save as white
    blankImage = np.zeros((height,width), dtype=np.uint8)
    fullImage = 255*np.ones((height,width),dtype=np.uint8)
    cv.imwrite(directory + "\\blankImage.png", blankImage)
    cv.imwrite(directory + "\\fullImage.png", fullImage)
        
    return pattern, offset


def convert_gray_code_to_decimal(gray_code_patterns):
    height, width, num_bits = gray_code_patterns.shape
    # num_bits -= 1 ### DELETE THIS AFTER TESTING
    binary_patterns = np.zeros((height, width, num_bits), dtype=np.uint8)
    binary_patterns[:, :, 0] = gray_code_patterns[:, :, 0]
    for i in range(1, num_bits):
        binary_patterns[:, :, i] = np.bitwise_xor(binary_patterns[:, :, i-1], gray_code_patterns[:, :, i])
        
    decimal_values = np.zeros((height, width), dtype=int)
    for i in range(num_bits):
        decimal_values += (2 ** (num_bits - 1 - i)) * binary_patterns[:, :, i]
    decimal_values += 1
    decimal_values -= int(64) # adjust decimal values according to aspect ratio to get columns 1 through 1920 (should be 64 for 1920 by 1080, 224 for 1600 by 1200). 
    return decimal_values 

In [3]:

def calculate_3D_points(decoded_image, camera_matrix, R, T, projector_intrinsics):
    height, width = decoded_image.shape
    points_3D = np.zeros((height, width, 3))

    # Define the normal to the projector planes (pointing in the x direction)
    plane_normal = np.array([1, 0, 0])
    points_3D = []
    # Iterate over each pixel in the decoded image
    for i in range(height):
        for j in range(width):
            projector_column = decoded_image[i, j]
            if projector_column <= 0:
                continue  # Skip invalid points

            # Image point in pixel coordinates
            u = np.array([j, i])  # 2D pixel coordinates

            # Convert the projector column index to world coordinatess
            projector_point_homogeneous = np.linalg.inv(projector_intrinsics).dot(np.array([projector_column, 0, 1]))
            projector_point_3D = projector_point_homogeneous[:3] / projector_point_homogeneous[2]
            plane_distance = projector_point_3D[0]  # Assuming distance is along the x-axis

            # Calculate the intersection point using the ray_to_plane_intersection function
            intersection_point = ray_to_plane_intersection(u, camera_matrix, R, T, plane_normal, plane_distance)
            if np.any(intersection_point): points_3D.append(intersection_point)

    return points_3D



def ray_to_plane_intersection(u, camera_matrix, R, T, plane_normal, plane_distance):

    # Convert image point to normalized camera coordinates
    u_normalized = np.linalg.inv(camera_matrix).dot(np.append(u, 1))

    # Calculate q (center of projection in world coordinates)
    q = -np.dot(R.T, T)

    # Calculate v (direction vector in world coordinates)
    v = np.dot(R.T, u_normalized)

    # Define the plane equation: n * (X - d * n) = 0
    # Calculate the intersection point
    lambda_val = (plane_distance - np.dot(plane_normal, q)) / np.dot(plane_normal, v)
    intersection_point = q + lambda_val * v

    return intersection_point


In [4]:

def pbpthreshold(image,threshold):
    """
    :param image: Input image.
    :param threshold: Threshold image.
    """
    #Compares image to thresholding image, set binary, if < than threshold image pixel = 0, if >, pixel goes to 255.
    flatimage = image.flatten()
    flatthreshold = threshold.flatten()
    if len(image) != len(threshold):
        print("Image and Threshold Matrix are incompatible sizes")
        return
    for i in range(len(flatimage)):
        if flatimage[i] <= flatthreshold[i]:
            flatimage[i] = 0
        elif flatimage[i] > flatthreshold[i]:
            flatimage[i] = 1
            
    image = flatimage.reshape(image.shape)
    return image

In [5]:
import open3d as o3d

def visualize_point_cloud(points_3D):

    # Convert the points to an open3d point cloud object
    point_cloud = o3d.geometry.PointCloud()
    point_cloud.points = o3d.utility.Vector3dVector(points_3D)
    o3d.visualization.draw_geometries([point_cloud])


In [6]:

# Camera intrinsic matrix for a simple pinhole camera model
focal_length = 1000
camera_matrix = np.array([[focal_length, 0, 500],
                          [0, focal_length, 500],
                          [0, 0, 1]])

projector_matrix = np.array([[focal_length, 0, 500],
                             [0, focal_length, 500],
                             [0, 0, 1]])


# Camera extrinsic parameters - assuming aligned with the world coordinates
R = np.eye(3)  # Identity matrix for rotation
T = np.array([-1000, 0, 0])  # Translation along Z-axis

# Simulated decoded image from structured light system
decoded_image = np.tile(np.arange(10), (1000, 1))  # 1000x1000 image with values from 0 to 9

# Calculate the 3D points
points_3D = calculate_3D_points(decoded_image, camera_matrix, R, T, projector_matrix)

# Check a few points
  # Print 3D points for a few columns at the middle row
visualize_point_cloud(points_3D)

In [7]:

width = 1600
height = 1200
grayPattern, offsets = generate_gray_code_patterns(width, height)

blankImage = cv.imread("testImages/blank_image.png")
fullImage = cv.imread("testImages/full_image.png") 

blankImage = cv.cvtColor(blankImage, cv.COLOR_BGR2GRAY)/255
fullImage = cv.cvtColor(fullImage,cv.COLOR_BGR2GRAY)/255
image_array = np.empty((grayPattern.shape[2],fullImage.shape[0],fullImage.shape[1]), dtype=grayPattern.dtype)
avg_thresh = cv.addWeighted(blankImage,0.5,fullImage,0.5,0) #add white and blank images for thresholding and average



# Viewing the patterns
# cv2.imshow('Pattern', grayPattern[:,:,0] * 255)
# cv2.waitKey(0)
# cv2.destroyAllWindows()



In [8]:

for i in range(grayPattern.shape[2]-1):

    # load image array and pre-process images
    filein = "testImages/image_{}.png".format(i+1)
    image = cv.imread(filein)
    image_gray = cv.cvtColor(image, cv.COLOR_BGR2GRAY)/255
    #Convert to black and white based on grascale (average per pixel thresholding)
    image_thresh = pbpthreshold(image_gray,avg_thresh)
    image_array[i] = image_thresh
    captured_patterns = np.transpose(image_array,(1,2,0)) #reorder shape for binary decoding 

# cv.imshow('book',image_array[4]*255)
# cv.waitKey(0)
# Convert Gray code to decimal
# col_values = convert_gray_code_to_decimal(captured_patterns)
# print(col_values)
# col_values = convert_gray_code_to_decimal(grayPattern)
# print(col_values)
# decoded_image = convert_gray_code_to_decimal(captured_patterns)


# K = np.array([[1642.17076,0,1176.14705],[0, 1642.83775, 714.90826],[0,0,1]])
# R = np.array([[1,0,0],[0,1,0],[0,0,1]])
# t = np.array([[0],[0],[1]])
# P_proj = np.array([[1642.17076,0,0],[0,1642.83775,0],[0,0,1]])

In [9]:
## Create an image mask from the full Image to block out shadows and non projected regions



def overlay_mask_with_threshold(image1, image2, threshold=10):
  
    # Ensure the images are the same size
    if image1.shape != image2.shape:
        raise ValueError("Images must have the same dimensions")

    # Compute the absolute difference between the images
    diff = cv.absdiff(image1, image2)

    # Convert difference to grayscale
    gray_diff = cv.cvtColor(diff, cv.COLOR_BGR2GRAY)

    # Create a mask where the difference is below the threshold (similar pixels)
    _, mask = cv.threshold(gray_diff, threshold, 255, cv.THRESH_BINARY_INV)

    # Create a red overlay
    red_overlay = np.zeros_like(image1, image1.dtype)
    red_overlay[:, :] = [0, 0, 255]  # Red color

    # Apply the mask to the red overlay
    red_mask = cv.bitwise_and(red_overlay, red_overlay, mask=mask)

    # Overlay the red mask on the original image
    overlay_result = cv.addWeighted(image1, 1, red_mask, 1, 0)

    # Display the result
    cv.imshow('Red Overlay on Image with Threshold', overlay_result)
    cv.waitKey(0)
    cv.destroyAllWindows()

    return mask

    # Optionally, save the result
    # cv2.imwrite('overlay_threshold_result.jpg', overlay_result)

# Example usage
# blankImage = cv.imread("C:\\Users\\nludw\\Documents\\Capstone\\Binary Coding\\Testing\\TestImages\\blankImage.png")
# fullImage = cv.imread("C:\\Users\\nludw\\Documents\\Capstone\\Binary Coding\\Testing\\TestImages\\fullImage.png") 
# mask = overlay_mask_with_threshold(fullImage, blankImage, threshold=5)
# print(mask.shape)


In [10]:
directory = "testImages/"
width = 1920
height = 1080
grayPattern, offsets = generate_gray_code_patterns(width, height)

blankImage = cv.imread(directory+"blank_image.png")
fullImage = cv.imread(directory+"full_image.png")
blankImage = cv.cvtColor(blankImage, cv.COLOR_BGR2GRAY)/255
fullImage = cv.cvtColor(fullImage,cv.COLOR_BGR2GRAY)/255
image_array = np.empty((grayPattern.shape[2],fullImage.shape[0],fullImage.shape[1]), dtype=grayPattern.dtype)
avg_thresh = cv.addWeighted(blankImage,0.5,fullImage,0.5,0) #add white and blank images for thresholding and average


for i in range(grayPattern.shape[2]-1):

    # load image array and pre-process images
    filein = "image_{}.png".format(i+1)
    image = cv.imread(directory+filein)
    image_gray = cv.cvtColor(image, cv.COLOR_BGR2GRAY)/255
    #Convert to black and white based on grascale (average per pixel thresholding)
    image_thresh = pbpthreshold(image_gray,avg_thresh)
    image_array[i] = image_thresh
    captured_patterns = np.transpose(image_array,(1,2,0)) #reorder shape for binary decoding 
    

In [11]:
blankImage = cv.imread(directory+"blank_image.png")
fullImage = cv.imread(directory+"full_image.png")
mask = overlay_mask_with_threshold(fullImage, blankImage, threshold=50)
print(mask.shape)

(1080, 1920)


In [ ]:


# K = np.array([[1642.17076,0,1176.14705],[0, 1642.83775, 714.90826],[0,0,1]])
# R = np.array([[1,0,0],[0,1,0],[0,0,1]])
# t = np.array([[0],[0],[1]])
# P_proj = np.array([[1642.17076,0,0],[0,1642.83775,0],[0,0,1]])
